# Data refinement

In [1]:
# Imports
import pandas as pd


In [30]:
# Load data. Right now we are using backup file, as we haven't scraped the full dataset.
df = pd.read_csv("C:\\Users\\Teemu\\Desktop\\Housing_market_analysis\\Housing-market-analysis-with-webscraped-data\\backups\\BACKUP-11.0K-iterations-Jun-29-2022-house-market-data.csv")
df.head()

,Unnamed: 0,Type,Address,District,City,Price (€),Area (sqm.),Year built,Form of ownership
0,0,Kerrostalo | 2h + k + p,Teekkarinkatu 19 A,Hervanta,Tampere,95 000,"60,5",1978,Omistusasunto
1,1,"Omakotitalo | 2(3)mh, oh, k, kk, 2xwc, kellari...",Jokiväylä 3,Korvanniemi,Rovaniemi,198 000,94,1953,Omistusasunto
2,2,Omakotitalo | 5 h + k + s + th,Tuomistontie 76,Jakkula,Laihia,48 000,"119,5",1982,Omistusasunto
3,3,"Kerrostalo | 2h, k, kph, piha",Kotinummenkuja 2,Tapanila,Helsinki,222 000,58,1985,Omistusasunto
4,4,"Kerrostalo | 1h, kk, kph",Huovitie 3,Haaga,Helsinki,137 000,"20,5",1956,Omistusasunto


In [31]:
# We can see that we have double index, so lets remove the column "Unnamed: 0".

df = df.drop(['Unnamed: 0'], axis= 1)
df

,Type,Address,District,City,Price (€),Area (sqm.),Year built,Form of ownership
0,Kerrostalo | 2h + k + p,Teekkarinkatu 19 A,Hervanta,Tampere,95 000,"60,5",1978,Omistusasunto
1,"Omakotitalo | 2(3)mh, oh, k, kk, 2xwc, kellari...",Jokiväylä 3,Korvanniemi,Rovaniemi,198 000,94,1953,Omistusasunto
2,Omakotitalo | 5 h + k + s + th,Tuomistontie 76,Jakkula,Laihia,48 000,"119,5",1982,Omistusasunto
3,"Kerrostalo | 2h, k, kph, piha",Kotinummenkuja 2,Tapanila,Helsinki,222 000,58,1985,Omistusasunto
4,"Kerrostalo | 1h, kk, kph",Huovitie 3,Haaga,Helsinki,137 000,"20,5",1956,Omistusasunto
...,...,...,...,...,...,...,...,...
10995,Kerrostalo | 1h + kt,Peltisepänkatu 10,Kesämäki,Lappeenranta,109 000,25,2019,Omistusasunto
10996,"Kerrostalo | 1h, kk, kph",Aleksanterinkatu 31 C,Keskusta,Tampere,182 000,32,1959,Omistusasunto
10997,Omakotitalo | 4 h + k + psh + s + at,Kämäränkatu 5,Könönpelto,Varkaus,74 000,98,1968,Omistusasunto
10998,Kerrostalo | 2h + kph + lasp + var,Tuusulantie 56,Kilta,Kerava,125 000,60,1990,Omistusasunto


In [33]:
# Lets check the data type of price and make sure they are numbers
print(df.dtypes)

Type                 object
Address              object
District             object
City                 object
Price (€)            object
Area (sqm.)          object
Year built           object
Form of ownership    object
dtype: object
